In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tfd = tf.contrib.distributions
tfb = tfd.bijectors
from tensorflow.keras import layers


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
tf.enable_eager_execution()

## Settings

In [3]:
DTYPE=tf.float32
NP_DTYPE=np.float32

In [756]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
QMat = np.random.rand(3,5)
# normalize the piecewise PDF such that the integral is 1
QMat = tf.nn.softmax(QMat)
# Insert a 0 to the beginning to match the size to the bin array
QMat = np.insert(QMat,0,0,axis=1)
# Print out values of Q for testing purposes
QMat

array([[0.        , 0.17694308, 0.12909111, 0.22869545, 0.22170349,
        0.24356687],
       [0.        , 0.23328833, 0.25628785, 0.13661589, 0.1210035 ,
        0.25280443],
       [0.        , 0.18656903, 0.13401585, 0.14672721, 0.22498145,
        0.30770645]])

In [781]:
# network in the coupling layer
def net(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return QMat
    return tf.softmax(layers.stack(x, layers.full_connected, [512, 512, out_size]))

class PiecewiseLinear(tfb.Bijector):
    """
    Piecewise Linear: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseLinear, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        self.width = 1.0/self.nbins
        
    def Q(self, xd):
        with tf.variable_scope('Q%d' % self.id, reuse=tf.AUTO_REUSE):
            return net(xd, self.D - self.d, self.nbins)
        
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.concat([xd, Q[np.arange(len(Q)),ibins]], axis=1)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        yD = (xD*self.nbins-ibins)*Q[np.arange(len(Q)),ibins+1]+np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins]
        return tf.concat([xd, yD], axis=1)
        
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(Q,axis=1),tf.transpose(yD),side='right'))-1
        xD = ((yD-np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins])*tf.reciprocal(Q[np.arange(len(Q)),ibins+1])+np.array(ibins,dtype=np.float32))*self.width
        return tf.concat([yd, xD], axis=1)
    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(Q,axis=1),tf.transpose(yD),side='right'))-1
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [782]:
test = PiecewiseLinear(6,3,5)

In [783]:
forward = test._forward(np.array([[0.1,0.2,0.3,0.4,0.5,0.6],[0.6,0.5,0.4,0.0,0.2,0.1]]).reshape(2,6))
forward

<tf.Tensor: id=9350, shape=(2, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.30603419, 0.55788413,
        0.4673121 ],
       [0.6       , 0.5       , 0.4       , 0.        , 0.23328833,
        0.09328452]])>

In [784]:
test._inverse(forward)

<tf.Tensor: id=9404, shape=(2, 6), dtype=float64, numpy=
array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
       [0.6, 0.5, 0.4, 0. , 0.2, 0.1]])>

In [785]:
inverse = test._inverse(np.array([0.17224058,0.17224058,0.17224058,0.17224058,0.17224058,0.17224058]).reshape(1,6))
inverse

<tf.Tensor: id=9451, shape=(1, 6), dtype=float64, numpy=
array([[0.17224058, 0.17224058, 0.17224058, 0.19468473, 0.14766326,
        0.18464005]])>

In [786]:
test._forward(inverse)

<tf.Tensor: id=9473, shape=(1, 6), dtype=float64, numpy=
array([[0.17224058, 0.17224058, 0.17224058, 0.17224058, 0.17224058,
        0.17224058]])>

In [787]:
test.pdf(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=9478, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.12909111, 0.25628785,
        0.14672721]])>

In [788]:
test._forward_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=9483, shape=(1,), dtype=float64, numpy=array([-0.49955723])>

In [789]:
test._inverse_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=9519, shape=(1,), dtype=float64, numpy=array([0.49955723])>

In [790]:
WMat = np.random.rand(3,5)
WMat = tf.nn.softmax(WMat)
WMat = np.insert(WMat,0,0,axis=1)
WMat

array([[0.        , 0.29250357, 0.14247968, 0.17536833, 0.20988416,
        0.17976427],
       [0.        , 0.23481226, 0.24852722, 0.13220091, 0.19999276,
        0.18446685],
       [0.        , 0.21736813, 0.27865387, 0.13870695, 0.15994708,
        0.20532397]])

In [940]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
VMat = np.random.rand(3,6)
expV = tf.exp(VMat)
VDenom = np.sum(0.5*(expV[:,0:5]+expV[:,1:6])*WMat[:,1:6],axis=1,keepdims=True)
# normalize the piecewise PDF such that the integral is 1
VMat = np.true_divide(expV,VDenom)
VMat = np.insert(VMat,0,0,axis=1)
# Print out values of Q for testing purposes
print(np.cumsum((VMat[:,1:7]+VMat[:,0:6])*WMat/2.0,axis=1))
VMat

[[0.         0.32894271 0.50643136 0.69998118 0.86090807 1.        ]
 [0.         0.25404276 0.52529236 0.64645992 0.83641251 1.        ]
 [0.         0.30678593 0.54008939 0.66455499 0.8168353  1.        ]]


array([[0.        , 1.17823131, 1.07092221, 1.42050187, 0.78685014,
        0.74663279, 0.80085974],
       [0.        , 1.08358087, 1.08021388, 1.10264233, 0.73043977,
        1.16915488, 0.60446994],
       [0.        , 1.91811224, 0.90461925, 0.76988437, 1.02477128,
        0.87936234, 0.90479077]])

In [1045]:
# network in the coupling layer
def netV(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return VMat

# network in the coupling layer
def netW(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return WMat

class PiecewiseQuadratic(tfb.Bijector):
    """
    Piecewise Quadratic: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseQuadratic, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        self.range = np.arange(self.d)
        
    def W(self, xd):
        with tf.variable_scope('W%d' % self.id, reuse=tf.AUTO_REUSE):
            return netW(xd, self.D - self.d, self.nbins)
              
    def V(self, xd):
        with tf.variable_scope('V%d' % self.id, reuse=tf.AUTO_REUSE):
            return netV(xd, self.D - self.d, self.nbins)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        WSum = np.cumsum(W,axis=1)
        VSum = np.cumsum((V[:,1:]+V[:,0:-1])*W/2.0,axis=1)
        ibins = tf.transpose(tf.searchsorted(WSum,tf.transpose(xD),side='right'))
        print(ibins)
        alpha = (xD-WSum[self.range,ibins-1])*tf.reciprocal(W[self.range,ibins])
        yD = alpha**2/2.0*np.diff(V)[self.range,ibins]*W[self.range,ibins] \
           + alpha*V[self.range,ibins]*W[self.range,ibins] \
           + VSum[self.range,ibins-1]
        return tf.concat([xd, yD], axis=1)
    
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        W = self.W(yd)
        V = self.V(yd)
        WSum = np.cumsum(W,axis=1)
        VSum = np.cumsum((V[:,1:]+V[:,0:-1])*W/2.0,axis=1)
        ibins = tf.transpose(tf.searchsorted(VSum,tf.transpose(yD),side='right'))
        print(ibins)
        denom = np.diff(V)[self.range,ibins]
        beta = (yD - VSum[self.range,ibins-1])/(W[self.range,ibins])
        print(yD)
        print(VSum[self.range,ibins-1])
        print(W[self.range,ibins])
        print(beta)
        xD = W[self.range,ibins]/denom*(-V[self.range,ibins]+tf.sqrt(V[self.range,ibins]**2+2*beta))+WSum[self.range,ibins-1]
        return tf.concat([yd, xD], axis=1)

In [1046]:
test = PiecewiseQuadratic(6,3,5)

In [1047]:
forward=test._forward(np.array([0.1,0.2,0.3,0.01,0.5,0.6]).reshape(1,6))
forward

tf.Tensor([[1 3 3]], shape=(1, 3), dtype=int32)


<tf.Tensor: id=15489, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.01176397, 0.54327221,
        0.63007392]])>

In [1048]:
test._inverse(forward)

tf.Tensor([[1 3 3]], shape=(1, 3), dtype=int32)
tf.Tensor([[0.01176397 0.54327221 0.63007392]], shape=(1, 3), dtype=float64)
[[0.         0.52529236 0.54008939]]
[[0.29250357 0.13220091 0.13870695]]
tf.Tensor([[0.04021821 0.13600402 0.64873849]], shape=(1, 3), dtype=float64)


<tf.Tensor: id=15567, shape=(1, 6), dtype=float64, numpy=
array([[ 0.1       ,  0.2       ,  0.3       , -0.09173358,  0.441739  ,
         0.82523565]])>

In [ ]:
            
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(W,axis=1),tf.transpose(xD),side='right')-1)
        ibinsp1 = ibins+1
        alpha = (xd-np.cumcum(W,axis=1)[np.arange(len(W)),ibins])/W[np.arange(len(W)),ibins]
        result = (V[np.arange(len(V)),ibinsp1]-V[np.arange(len(V)),ibins])*alpha+V[np.arange(len(V)),ibins]
        return tf.concat([xd, result], axis=1)       

    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = np.array(np.floor(yD*self.nbins),dtype=np.int32)
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [797]:
np.diff(VMat)

array([[-0.03252278,  0.05813606,  0.91280746, -0.46959701,  0.30203176],
       [-0.60856429,  0.1897924 ,  0.60831937, -0.77393558,  0.31171796],
       [-0.60916156, -0.09294806,  0.65000014, -0.37539562, -0.32373172]])

In [250]:
VMat

array([[0.95857261, 0.89965627, 1.01485801, 1.02914925, 1.14464312,
        0.94965425],
       [0.83384493, 1.02240254, 1.2870976 , 0.90594321, 0.7476643 ,
        1.24475353],
       [0.78848073, 0.9873137 , 1.30597736, 0.5866991 , 1.33639998,
        0.88931847]])

In [314]:
tf.gather_nd(WMat,[[0,1],[1,3],[2,4]])

<tf.Tensor: id=4101, shape=(3,), dtype=float64, numpy=array([0.13723821, 0.21514902, 0.19268482])>

In [315]:
WMat

<tf.Tensor: id=3191, shape=(3, 5), dtype=float64, numpy=
array([[0.30461836, 0.13723821, 0.19126584, 0.1497618 , 0.21711578],
       [0.21034329, 0.17996762, 0.25946607, 0.21514902, 0.135074  ],
       [0.23739297, 0.16027006, 0.18968912, 0.21996303, 0.19268482]])>